In [176]:
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import Counter, defaultdict
import pandas as pd
import pickle
import numpy as np
from shapely.geometry import Polygon, Point, LinearRing, box
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction import DictVectorizer
%matplotlib notebook

In [44]:
df_train = pd.read_pickle('df_train')
df_test = pd.read_pickle('df_test')
train = df_train[df_train['duration'] != 0.0]
test = df_test[(df_test['duration'] != 0.0) & (df_test.x_exit.isnull()) & (df_test.y_exit.isnull())]
test_start_not_in_city = test[~(((test['x_entry'] >= 3750901.5068) &
         (test['x_entry'] <= 3770901.5068)) & ((test['y_entry'] >= -19268905.6133) &
         (test['y_entry'] <= -19208905.6133)))]
test_start_in_city = test[((test['x_entry'] >= 3750901.5068) &
         (test['x_entry'] <= 3770901.5068)) & ((test['y_entry'] >= -19268905.6133) &
         (test['y_entry'] <= -19208905.6133))]
train_not_in_city_1 = train[(~((train['x_entry'] >= 3750901.5068) &
         (train['x_entry'] <= 3770901.5068) & (train['y_entry'] >= -19268905.6133) &
         (train['y_entry'] <= -19208905.6133))&(train['label'] == 1))]
train_not_in_city_0 = train[(~((train['x_entry'] >= 3750901.5068) &
         (train['x_entry'] <= 3770901.5068) & (train['y_entry'] >= -19268905.6133) &
         (train['y_entry'] <= -19208905.6133))&(train['label'] == 0))]
train_in_city_0 = train[(((train['x_entry'] >= 3750901.5068) &
         (train['x_entry'] <= 3770901.5068) & (train['y_entry'] >= -19268905.6133) &
         (train['y_entry'] <= -19208905.6133))&(train['label'] == 0))]
train_in_city_1 = train[(((train['x_entry'] >= 3750901.5068) &
         (train['x_entry'] <= 3770901.5068) & (train['y_entry'] >= -19268905.6133) &
         (train['y_entry'] <= -19208905.6133))&(train['label'] == 1))]

In [3]:
min_x_entry = min(min(df_train.x_entry.describe()['min'], df_test.x_entry.describe()['min']),
                  df_train.x_exit.describe()['min'])
min_y_entry = min(min(df_train.y_entry.describe()['min'], df_test.y_entry.describe()['min']),
                  df_train.y_exit.describe()['min'])
max_x_entry = max(max(df_train.x_entry.describe()['max'], df_test.x_entry.describe()['max']),
                  df_train.x_exit.describe()['max'])
max_y_entry = max(max(df_train.y_entry.describe()['max'], df_test.y_entry.describe()['max']),
                  df_train.y_exit.describe()['max'])
print(min_x_entry, max_x_entry, min_y_entry, max_y_entry)

3740998.35481912 3777099.2656833767 -19382914.9809002 -19042656.658487003


In [26]:
def plot_map():
    rect_all = patches.Rectangle((3740998.35481912, -19042656.658487003), (3777099.2656833767 - 3740998.35481912),
                             (- 19382914.9809002 + 19042656.658487003),
                             linewidth=1,edgecolor='g',fill = False,hatch = '\\\\\\', label = 'city')
    city = plt.gca().add_patch(rect_all)
    rect_city = patches.Rectangle((3750901.5068, -19208905.6133), (3770901.5068 - 3750901.5068),
                             (- 19268905.6133 + 19208905.6133),
                             linewidth=1,edgecolor='r',fill = False,hatch = '//////', label = 'city center')
    city_center = plt.gca().add_patch(rect_city)
    plt.legend(handles=[city, city_center])
    plt.grid(True)
#     plt.xlim(min_x_entry, max_x_entry)
    plt.ylim(min_y_entry, max_y_entry)
    plt.xlim(3740000, 3780000)
    plt.ylim(-19000000, - 19400000)
    plt.show()

In [127]:
def get_box(x,y):
    n1 = -1
    n2 = -1
    if(~np.isnan(x) and ~np.isnan(y)):
        if(x >= 3740000.0000000000 and x < 3745000.0000000000):
            n1 = 1
        elif(x >= 3745000.0000000000 and x < 3750000.0000000000):
            n1 = 2
        elif(x >= 3750000.0000000000 and x < 3755000.0000000000):
            n1 = 3
        elif(x >= 3755000.0000000000 and x < 3760000.0000000000):
            n1 = 4
        elif(x >= 3760000.0000000000 and x < 3765000.0000000000):
            n1 = 5
        elif(x >= 3765000.0000000000 and x < 3770000.0000000000):
            n1 = 6
        elif(x >= 3770000.0000000000 and x < 3775000.0000000000):
            n1 = 7
        else:
            n1 = 8

        if(y >= -19400000.0000000000 and y < -19350000.0000000000):
            n2 = 8
        elif(y >= -19350000.0000000000 and y < -19300000.0000000000):
            n2 = 7
        elif(y >= -19300000.0000000000 and y < -19250000.0000000000):
            n2 = 6
        elif(y >= -19250000.0000000000 and y < -19200000.0000000000):
            n2 = 5
        elif(y >= -19200000.0000000000 and y < -19150000.0000000000):
            n2 = 4
        elif(y >= -19150000.0000000000 and y < -19100000.0000000000):
            n2 = 3
        elif(y >= -19100000.0000000000 and y < -19050000.0000000000):
            n2 = 2
        else:
            n2 = 1
    
        return (n2-1)*8 + n1
    else:
        return 0

In [123]:
get_box(df_train.iloc[500].x_exit, df_train.iloc[500].y_exit)

36

In [11]:
df_train['entry_box'] = df_train.apply(lambda row: get_box(row['x_entry'], row['y_entry']) , axis=1)
df_train['exit_box'] = df_train.apply(lambda row: get_box(row['x_exit'], row['y_exit']) , axis=1)
df_test['entry_box'] = df_test.apply(lambda row: get_box(row['x_entry'], row['y_entry']) , axis=1)
df_test['exit_box'] = df_test.apply(lambda row: get_box(row['x_exit'], row['y_exit']) , axis=1)

In [129]:
df_train.to_pickle('df_train')
df_test.to_pickle('df_test')

In [102]:
df = df_train.groupby('hash')
df_list = list(df)

In [191]:
df_test = df_test.groupby('hash')
df_test_list = list(df_test)

In [190]:
dataset = []
print(len(df_list))
for index, sample in enumerate(df_list):
    length = sample[1].shape[0]
    count = 0
    features = {}
    features['hash'] = sample[0]
    for index1, row in sample[1].iterrows():
        if(count < length - 1):
            features['entry_'+ str(row['entry_box'])]  = features.get('entry_'+ str(row['entry_box']),0) + 1
            features['exit_' + str(row['exit_box'])]  = features.get('exit_'+ str(row['exit_box']),0) + 1
        else:
            features['l_entry']  = features.get('entry_'+ str(row['entry_box']),0) + 1
            features['l_exit']  = features.get('exit_'+ str(row['exit_box']),0) + 1
            features['x_entry'] = row['x_entry']
            features['y_entry'] = row['y_entry']
            features['duration'] = row['duration']
            features['city_distance'] = row['city_distance']
            features['hour'] = row['hour']
            features['label'] = row['label']
        count = count + 1
    dataset.append(features)

134063


In [198]:
dataset = []
print(len(df_list))
for index, sample in enumerate(df_list):
    length = sample[1].shape[0]
    count = 0
    features = {}
    features['hash'] = sample[0]
    for index1, row in sample[1].iterrows():
        if(count < length - 1):
            features['entry_'+ str(row['entry_box'])]  = features.get('entry_'+ str(row['entry_box']),0) + 1
            features['exit_' + str(row['exit_box'])]  = features.get('exit_'+ str(row['exit_box']),0) + 1
        else:
            features['l_entry']  = row['entry_box']
            features['l_exit']  = row['exit_box']
            features['x_entry'] = row['x_entry']
            features['y_entry'] = row['y_entry']
            features['duration'] = row['duration']
            features['city_distance'] = row['city_distance']
            features['hour'] = row['hour']
            features['label'] = row['label']
        count = count + 1
    dataset.append(features)

134063


In [223]:
test_dataset = []
print(len(df_test_list))
for index, sample in enumerate(df_test_list):
    length = sample[1].shape[0]
    count = 0
    features = {}
    features['hash'] = sample[0]
    for index1, row in sample[1].iterrows():
        features['trajectory_id']  = row['trajectory_id']
        if(count < length - 1):
            features['entry_'+ str(row['entry_box'])]  = features.get('entry_'+ str(row['entry_box']),0) + 1
            features['exit_' + str(row['exit_box'])]  = features.get('exit_'+ str(row['exit_box']),0) + 1
        else:
            features['l_entry']  = row['entry_box']
            features['l_exit']  = 0
            features['x_entry'] = row['x_entry']
            features['y_entry'] = row['y_entry']
            features['duration'] = row['duration']
            features['city_distance'] = row['city_distance']
            features['hour'] = row['hour']
            features['label'] = -1
        count = count + 1
    test_dataset.append(features)

33515


In [200]:
x_train_df = pd.DataFrame(dataset).fillna(0)

In [228]:
x_test_df = pd.DataFrame(test_dataset).fillna(0)
x_test_df['entry_5'] = 0.0
x_test_df['exit_5'] = 0.0

In [229]:
print(x_train_df.shape, x_test_df.shape)

(134063, 117) (33515, 118)


In [230]:
training_dataset = x_train_df.drop(['hash','l_exit'], axis=1)
testing_dataset = x_test_df.drop(['hash','l_exit','trajectory_id'], axis=1)

In [216]:
x_1 = training_dataset.drop(['label'], axis=1).values
y_1 = training_dataset['label'].values
sc_1 = StandardScaler()
x_1 = sc_1.fit_transform(x_1)

In [217]:
xTrain, xTest, yTrain, yTest = train_test_split(x_1, y_1, test_size = 0.1, random_state = 0)
clf_1 = RandomForestClassifier(n_jobs = -1, n_estimators=100,max_depth = 40, random_state=42)
clf_1.fit(xTrain, yTrain)
yPred = clf_1.predict(xTest)
print(classification_report(yTest, yPred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      9768
           1       0.92      0.85      0.89      3639

   micro avg       0.94      0.94      0.94     13407
   macro avg       0.93      0.91      0.92     13407
weighted avg       0.94      0.94      0.94     13407



In [218]:
for feature in zip(yTest, clf_1.feature_importances_):
    print(feature)

(0, 0.31707452389274526)
(0, 0.07744485114852785)
(0, 0.0013352544897161276)
(0, 0.001061261846434303)
(1, 0.0013125000812986743)
(0, 0.0008574758833072347)
(0, 0.001538917480411402)
(0, 7.797089529127445e-05)
(0, 0.00024800250373864204)
(1, 0.0011914778359290205)
(1, 0.0010784444631022777)
(0, 0.001248215540263061)
(1, 0.0011847508530661599)
(1, 0.001461443158320305)
(1, 0.0008599214718489718)
(0, 0.0006276772012806657)
(0, 0.0011221537552791804)
(0, 0.0020776717479500297)
(0, 0.0017252146710061464)
(0, 1.023530019889496e-05)
(0, 0.0025162298802622555)
(0, 0.0011462775189435815)
(0, 0.0013851845551886674)
(0, 0.001368017360578957)
(0, 0.001331528261030952)
(1, 0.0033922495866112335)
(0, 0.035549152729202886)
(1, 0.021660060237046464)
(0, 0.008001012087572474)
(0, 0.0021008392670142863)
(1, 0.00020936545965460223)
(0, 0.0018171205518373657)
(1, 0.0031392381470791387)
(1, 0.0031201268541063633)
(0, 0.0021581337466734336)
(1, 0.0051456164282722)
(1, 0.004479671655299749)
(1, 0.0023717216

In [240]:
x_test_df['label'] =  clf_1.predict(sc_1.fit_transform(testing_dataset.drop(['label'], axis=1).values))

In [241]:
x_test_df['label'] = x_test_df.apply(lambda row: 0 
                                 if ((row['duration'] == 0.0) &
                                     ~(((row['x_entry'] >= 3750901.5068) &
                                                               (row['x_entry'] <= 3770901.5068)) &
                                       ((row['y_entry'] >= -19268905.6133) &
                                        (row['y_entry'] <= -19208905.6133))))
                                 else row['label'], axis=1)
x_test_df['label'] = x_test_df.apply(lambda row: 1
                                 if ((row['duration'] == 0.0) &
                                     (((row['x_entry'] >= 3750901.5068) &
                                                               (row['x_entry'] <= 3770901.5068)) &
                                       ((row['y_entry'] >= -19268905.6133) &
                                        (row['y_entry'] <= -19208905.6133))))
                                 else row['label'], axis=1)

In [242]:
x_test_df.label.value_counts()

0    25205
1     8310
Name: label, dtype: int64

In [233]:
x_test_df.label.value_counts()

0    25168
1     8347
Name: label, dtype: int64

In [243]:
x_test_df.sort_values('trajectory_id',ascending=True)[['trajectory_id','label']].to_csv('output23.csv', index=False)